In [22]:
from sqlite3 import DatabaseError, IntegrityError
from typing import List

from sqlalchemy import update, insert
from sqlalchemy.orm import Session

from data_vortex.rightmove_models import RightmoveRentalListing, Price, Currency, PriceUnit
from data_vortex.database.models import RentalListing

In [2]:
def create_listing(db: Session, rental_listing: RightmoveRentalListing):
    listing_dict = rental_listing.to_orm_dict()
    try:
        existing_listing = (
            db.query(RentalListing)
            .filter(RentalListing.property_id == rental_listing.property_id)
            .first()
        )
        if existing_listing:
            # return RightmoveRentalListing(**existing_listing.dict())
            pass
        else:
            db.execute(
                insert(RentalListing),
                listing_dict,
            )
    except IntegrityError as e:
        db.rollback()
        raise ValueError(f"Integrity error: {e!s}")
    except DatabaseError as e:
        db.rollback()
        raise Exception(f"Database error: {e!s}")

In [13]:
from data_vortex.database.database import SessionLocal, create_database

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()
        
db = next(get_db())  # Get a session

In [14]:
existing_listing = (
    db.query(RentalListing)
)

In [15]:
existing_listing.first()

In [25]:
nl = RightmoveRentalListing(
    property_id="123456",
    description="description",
    price=Price(price=100, currency=Currency.GBP, per=PriceUnit.PER_WEEK),
    added_date="2020-02-01",
    address="123 Hornsey Lane Gardens, N6 5NX, London",
    postcode="N6 5NX",
)

In [26]:
create_listing(db, rental_listing=nl)